# Trace file analysis
This Jupyter notebook can be used to analyze the trace files generated by the Analog Discovery measurement device.<br>
**Input:** Current trace files with the following format:
```csv
time_S,signal_V,trigger_V
0.0,0.04344940590134493,0.0005593676687323445
2e-06,0.04747404468672084,0.0012284272423136715
4e-06,0.049821750644856784,0.000893897455523008
```

**Output:** 
Calculated energy consumption, duration and the corresponding standard deviations.

In [ ]:
import pandas as pd
from scipy import integrate
import os

## Configuration

In [ ]:
base_dir = "/Users/thomas/Documents/RP2040/"

In [ ]:
R_shunt = 1
V_SS = 3.3

In [ ]:
def calc_AmpereSecFromDataFrame(df):
    return integrate.trapezoid(df.current_A, df.time)

def calc_load_coulomb_str(df):
    load = calc_AmpereSecFromDataFrame(df)
    return "Q = {:.4f} As".format(load)

def get_model_execution_duration(df):
    return df.iloc[-1].time - df.iloc[0].time

In [ ]:
def analyze_trace_file(path):
    df = pd.read_csv(path, names=["time", "signal_V", "trigger", "current_A"], sep=",", skiprows=1)
    df.current_A = df.signal_V / R_shunt
    #print(df.current_A.mean())
    df_active = df[df.trigger > 2]
    duration = get_model_execution_duration(df_active)
    charge_As = calc_AmpereSecFromDataFrame(df_active)
    charge_Ah = charge_As / 3600
    energy_Wh = charge_Ah * V_SS
    energy_Joule = energy_Wh * 3600
    return (duration, energy_Joule)

In [ ]:
def analyze_subdirectory(measurement_dir_path):
    print("Measurement dir:", measurement_dir_path)
    df = pd.DataFrame(columns=["file", "energyJoule"])
    for filename in os.listdir(measurement_dir_path):
        if ".csv" not in filename:
            continue

        path = measurement_dir_path + filename
        (duration, energy_consumption) = analyze_trace_file(path)
        new_row = {'file':filename, 'energyJoule':energy_consumption, 'duration':duration}
        df = df.append(new_row, ignore_index=True)
        #print(path, "Energy consumption:", energy_consumption)

    return df


def analyze_directory(directory):
    dir_path = base_dir + directory + "/"
    df_total = pd.DataFrame()
    for CR in [1, 2, 3, 4, 8, 16, 24, 32, 48, 64]:
        CR_percent = 0
        if CR > 0:
            CR_percent = 1 - (1 / CR)
            
        measurement_dir_path = dir_path + str(CR) + "/"
        if not os.path.exists(measurement_dir_path):
            continue
        
        df = analyze_subdirectory(measurement_dir_path)

        print("CR:", CR, "Energy:", df.energyJoule.mean(), "Duration:", df.duration.mean())
        new_row = {'CR':CR, 'CRPercent': CR_percent, 'energyJoule':df.energyJoule.mean(), 'duration': df.duration.mean(), 'energyJouleStd': df.energyJoule.std(), 'durationStd': df.duration.std()}
        df_total = df_total.append(new_row, ignore_index=True)
    
    return df_total

## Trace file analysis

### Base-model

In [ ]:
df_baseline = analyze_directory("baseline")

### Unstructured pruning

In [ ]:
df_unstructured = analyze_directory("unstructured_pruning")

### Structured pruning

In [ ]:
df_structured = analyze_directory("structured_pruning")

### Unstructured pruning (speedup)

In [ ]:
df_unstructured_speedup = analyze_directory("unstructured_pruning_speedup")

In [ ]:
df_unstructured_with_baseline = df_baseline.append(df_unstructured, ignore_index=True)
df_structured_with_baseline = df_baseline.append(df_structured, ignore_index=True)

## Describe datasets

In [ ]:
df_unstructured_with_baseline.head(10)

In [ ]:
df_structured_with_baseline.head(10)

In [ ]:
df_unstructured_speedup.head(10)

## Export data

In [ ]:
df_unstructured_with_baseline.to_csv(path_or_buf="../../../src/mnist/train/models/unstructured_pruning/inference.csv", sep=';', index=None)
df_structured_with_baseline.to_csv(path_or_buf="../../../src/mnist/train/models/structured_pruning/inference.csv", sep=';', index=None)
df_unstructured_speedup.to_csv(path_or_buf="../../../src/mnist/train/models/unstructured_pruning/inference_speedup.csv", sep=';', index=None)